In [19]:
import sys
sys.path.append('../modules/')

from get_data import get_data
from random import sample
import re



X, y = get_data()
y = y.where(X.reviewText.str.contains(r'[a-zA-Z]'))
X = X.where(X.reviewText.str.contains(r'[a-zA-Z]'))

X = X.reviewText.str.replace('\n', ' ')
print(X)
ranges = [100, 500, 2000]

neg = X[y == 0]
pos = X[y == 1]



for r in ranges:
    print('\nProducing subset for size:', r)
#     text = []
    
    text_neg = sample(list(neg), r//2)
    text_neg = ["0"+x for x in text_neg]
    text_pos = sample(list(pos), r//2)
    text_pos = ["1"+x for x in text_pos]
    text = text_neg+text_pos
#     for label, data in enumerate([neg, pos]):
#         print("data:",len(data))
        
#         idxs = range(len(data))
#         idxs = sample(idxs, r//2)
#         print(idxs)
#         print(data.iloc[idxs])
#         data.iloc[idxs][""]
#         break
#         text += list(data.iloc[idxs].reviewText.str.replace('\n', ' '))
        
    pattern1 = re.compile(r'([^0-9a-zA-Z\s])\1+(?=[a-z0-9A-Z])')
    pattern2 = re.compile(r'([^0-9a-zA-Z\s])\1+')
    pattern3 = re.compile(r'<[^>]>')
    for i, t in enumerate(text):
        t = re.sub(pattern1, r'\1 ', t)
        t = re.sub(pattern2, r'\1', t)
        t = re.sub(pattern3, r'', t)
        text[i] = t


    # fix "...blabl -> ... blabl" and reduce "... -> ."
    s_before = len(text)
    text[:] = filter(lambda sen: len(sen) > 35, text)
    print('Removed short sentences:', s_before - len(text))

    s_before = len(text)
    new = []
    # Cut long sentences
    for review in text:
        shortened_sentences = 0
        if len(review) > 450:
            extra = review[450:]
            match = re.search(r'[!?.]', extra)
            if match:
                stop = 450 + match.span()[-1]
                review = review[:stop]
                shortened_sentences += 1
            if len(review) > 600:
                continue
        new.append(review)
#     print('Shortened long sentences:', shortened_sentences)
#     print('Removed difficult long sentences:', s_before - len(new))

    with open(f'../Data/subsets/n_{r}.txt', 'w') as f:
        for i, t in enumerate(text):
            l = t[0]
            t = t[1:]
            f.write(f'{l}\t{t}\n')

    with open(f'../Data/subsets/n_{r}_for_gpt.txt', 'w') as f:
        for i, t in enumerate(new):
            l = t[0]
            t = t[1:]
            f.write(f'{l}\t{t}\n')

0        So creative!  Love his music - the words, the ...
1        This tape can hardly be understood and it was ...
2        Buy the CD.  Do not buy the MP3 album.  Downlo...
3        I love Dallas Holms music and voice!  Thank Yo...
4               Great memories of my early years in Christ
                               ...                        
99995                             Great old song. Love it.
99996    Daniel Smith has no more command of the bassoo...
99997                    I'm enjoying her album very much!
99998    These digital files are corrupted.  There are ...
99999    I saw the Tempest live performed by the Minnes...
Name: reviewText, Length: 100000, dtype: object

Producing subset for size: 100
Removed short sentences: 18

Producing subset for size: 500
Removed short sentences: 107

Producing subset for size: 2000
Removed short sentences: 396


In [13]:
y

0        1.0
1        0.0
2        0.0
3        1.0
4        1.0
        ... 
99995    1.0
99996    0.0
99997    1.0
99998    0.0
99999    0.0
Name: sentiment, Length: 100000, dtype: float64

In [16]:
sample(list(neg["reviewText"]),3)

['Company/seller is wonderful, DVD is too orchestral and not what I expected at all. Seller very nice. Returned',
 "If you LIKE Disturbed, you will LIKE this CD.  If you do NOT LIKE Disturbed, you will NOT LIKE this CD.  I don't really like Disturbed.  Do the math.",
 'faulty']